# Проверка бандла в Microsoft Word

Загрузим Data Bundle

In [120]:
from tg.common import DataBundle

db = DataBundle.load(r'C:\Users\Bogdan\PycharmProjects\grammar_ru\demos\files\tsa-test.zip')

src = db.src
src

,word_id,sentence_id,word_index,paragraph_id,word_tail,word,word_type,word_length,file_id,corpus_id,is_target,label,original_corpus,original_word_id,original_sentence_id,original_paragraph_id,updated
0,0,0,0,0,1,—,punct,1,273a88ef-0d47-4a69-8c84-9b91e2d5427f,transfuzed.zip,False,0,pub.base.zip,68,4,2,False
1,1,0,1,0,1,Фильм,ru,5,273a88ef-0d47-4a69-8c84-9b91e2d5427f,transfuzed.zip,False,0,pub.base.zip,69,4,2,False
2,2,0,2,0,0,«,punct,1,273a88ef-0d47-4a69-8c84-9b91e2d5427f,transfuzed.zip,False,0,pub.base.zip,70,4,2,False
3,3,0,3,0,0,Растворяться,ru,12,273a88ef-0d47-4a69-8c84-9b91e2d5427f,transfuzed.zip,True,0,pub.base.zip,71,4,2,False
4,4,0,4,0,1,»,punct,1,273a88ef-0d47-4a69-8c84-9b91e2d5427f,transfuzed.zip,False,0,pub.base.zip,72,4,2,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66199,66199,66188,6,66188,1,будут,ru,5,273a88ef-0d47-4a69-8c84-9b91e2d5427f,transfuzed.zip,False,0,pub.base.zip,38468826,38468369,38468341,False
66200,66200,66188,7,66188,1,делиться,ru,8,273a88ef-0d47-4a69-8c84-9b91e2d5427f,transfuzed.zip,True,0,pub.base.zip,38468827,38468369,38468341,False
66201,66201,66188,8,66188,1,между,ru,5,273a88ef-0d47-4a69-8c84-9b91e2d5427f,transfuzed.zip,False,0,pub.base.zip,38468828,38468369,38468341,False
66202,66202,66188,9,66188,0,сторонами,ru,9,273a88ef-0d47-4a69-8c84-9b91e2d5427f,transfuzed.zip,False,0,pub.base.zip,38468829,38468369,38468341,False


Подключим клиент для работы с MS Word

In [121]:
from qa.enums.language import Language
from word_client import MSWordClient

client = MSWordClient(Language.RUSSIAN)

client.spell_check("Мне часто приходится боротся с трудностями")

[SpellError(word='боротся', index=3, start_position=21, end_position=28, suggestions=['борется', 'борются', 'бороться', 'берётся', 'берется', 'берутся', 'бурится', 'бурятся', 'боролся', 'боротая'])]

Получим список случайных идентификаторов предложений

In [122]:
import numpy as np


ALL_SENTENCES_COUNT = 100
NEGATIVE_SENTENCES_COUNT = 60

uniq_sentence_ids = np.random.choice(src['sentence_id'].unique(), ALL_SENTENCES_COUNT, replace=False)

Получим dataframe с корректными предложениями

In [123]:
positive_sentence_ids = uniq_sentence_ids[NEGATIVE_SENTENCES_COUNT:]

positive_src = src[src['sentence_id'].isin(positive_sentence_ids)]

positive_src.head()

,word_id,sentence_id,word_index,paragraph_id,word_tail,word,word_type,word_length,file_id,corpus_id,is_target,label,original_corpus,original_word_id,original_sentence_id,original_paragraph_id,updated
444,444,332,0,332,1,Они,ru,3,273a88ef-0d47-4a69-8c84-9b91e2d5427f,transfuzed.zip,False,1,pub.base.zip,132715,132292,132281,False
445,445,332,1,332,1,не,ru,2,273a88ef-0d47-4a69-8c84-9b91e2d5427f,transfuzed.zip,False,1,pub.base.zip,132716,132292,132281,False
446,446,332,2,332,1,ровня,ru,5,273a88ef-0d47-4a69-8c84-9b91e2d5427f,transfuzed.zip,False,1,pub.base.zip,132717,132292,132281,False
447,447,332,3,332,1,ни,ru,2,273a88ef-0d47-4a69-8c84-9b91e2d5427f,transfuzed.zip,False,1,pub.base.zip,132718,132292,132281,False
448,448,332,4,332,1,по,ru,2,273a88ef-0d47-4a69-8c84-9b91e2d5427f,transfuzed.zip,False,1,pub.base.zip,132719,132292,132281,False


Получим отрицательный dataframe с предложениями, где правильные глаголы на -тся/-ться заменены на ошибочные

In [124]:
negative_sentence_ids = uniq_sentence_ids[:NEGATIVE_SENTENCES_COUNT]
negative_src = src[src['sentence_id'].isin(negative_sentence_ids)]

def replace_endings(word):
    if word.endswith('тся'):
        return word[:-3] + 'ться'
    elif word.endswith('ться'):
        return word[:-4] + 'тся'
    else:
        return word

negative_src = negative_src.copy()
negative_src['word'] = negative_src['word'].apply(replace_endings)

negative_src.head()

,word_id,sentence_id,word_index,paragraph_id,word_tail,word,word_type,word_length,file_id,corpus_id,is_target,label,original_corpus,original_word_id,original_sentence_id,original_paragraph_id,updated
4104,4104,4104,0,4104,1,Автором,ru,7,273a88ef-0d47-4a69-8c84-9b91e2d5427f,transfuzed.zip,False,1,pub.base.zip,3099442,3099051,3099034,False
4105,4105,4104,1,4104,1,на,ru,2,273a88ef-0d47-4a69-8c84-9b91e2d5427f,transfuzed.zip,False,1,pub.base.zip,3099443,3099051,3099034,False
4106,4106,4104,2,4104,1,программке,ru,10,273a88ef-0d47-4a69-8c84-9b91e2d5427f,transfuzed.zip,False,1,pub.base.zip,3099444,3099051,3099034,False
4107,4107,4104,3,4104,1,значится,ru,8,273a88ef-0d47-4a69-8c84-9b91e2d5427f,transfuzed.zip,True,1,pub.base.zip,3099445,3099051,3099034,False
4108,4108,4104,4,4104,0,В,ru,1,273a88ef-0d47-4a69-8c84-9b91e2d5427f,transfuzed.zip,False,1,pub.base.zip,3099446,3099051,3099034,False


In [125]:
from pandas import DataFrame
from tg.grammar_ru import Separator


def get_errors_count(df: DataFrame) -> int:
    errors_count = 0

    for id in df['sentence_id'].unique():
        sentence = Separator.Viewer().to_text(df.loc[df.sentence_id == id])
        errors_count += len(client.spell_check(sentence, error_qualifier=TsaErrorQualifier))

    return errors_count

Получим число ошибок употребления -тся/-ться, которое найдет MS Word в предложениях из позитивного dataframe

In [126]:
positive_errors_count = get_errors_count(positive_src)
positive_errors_count

1

Получим число ошибок употребления -тся/-ться, которое найдет MS Word в предложениях из отрицательного dataframe

In [127]:
negative_errors_count = get_errors_count(negative_src)
negative_errors_count

12

In [129]:
true_positive = negative_errors_count # ошибка есть и она найдена
false_positive = positive_errors_count # ошибки нет, но она найдена
true_negative = positive_src[positive_src['is_target'] == True].shape[0] - positive_errors_count # ошибки нет и она не найдена
false_negative = negative_src[negative_src['is_target'] == True].shape[0] - negative_errors_count # ошибка есть, но она не найдена

true_positive, false_positive, true_negative, false_negative

(12, 1, 39, 51)

Посчитаем точность

In [130]:
precision = true_positive / (true_positive + false_positive)

precision

0.9230769230769231

Посчитаем полноту

In [131]:
recall = true_positive / (true_positive + false_negative)

recall

0.19047619047619047

Посчитаем f1

In [132]:
f1_score = 2 * precision * recall / (recall + precision)

f1_score

0.31578947368421056